In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd '/content/drive/My Drive/'

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
!pip install leidenalg igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import scipy.io as sio
import numpy as np
import igraph as ig
import leidenalg
from itertools import combinations
mat=sio.loadmat('A.mat')['A'] #Import the multiplex network of sparse kernel matrices
mat

array([[[0.19236087, 0.        , 0.        , ..., 1.        ,
         1.        , 1.        ],
        [0.0477099 , 0.        , 0.94778694, ..., 0.        ,
         0.        , 0.9954516 ],
        [0.1808883 , 0.        , 0.91084773, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.37005921, 0.        , 0.99999999, ..., 0.        ,
         0.79571821, 0.        ],
        [0.1393475 , 0.        , 0.89867883, ..., 0.        ,
         0.79571821, 0.97612508],
        [0.04795336, 0.        , 0.86654573, ..., 0.        ,
         0.83626018, 0.        ]],

       [[0.99928815, 0.        , 0.64942737, ..., 0.        ,
         0.        , 0.9954516 ],
        [0.78877981, 0.        , 0.        , ..., 1.        ,
         1.        , 1.        ],
        [0.99746165, 0.        , 0.        , ..., 0.74936216,
         0.8423295 , 0.        ],
        ...,
        [0.93455342, 0.        , 0.        , ..., 0.94783595,
         0.        , 0.        ],
        [0.9

In [ ]:
def Divisive_Clustering(X):
    """
    Consensus-based community detection for directed or undirected networks
    using the Leiden algorithm.

    Arguments:
    X : 3D numpy array of shape (N, N, L), where N is the number of nodes,
        and L is the number of layers (each representing a weighted adjacency matrix).

    Returns:
    Opt_rank : The optimal number of clusters to extract
    M : Community assignment of nodes
    Q : Modularity value of the partition
    aggregated_vs : Co-membership matrix aggregated across layers
    """
    # Validate input dimensions
    if len(X.shape) != 3 or X.shape[0] != X.shape[1]:
        raise ValueError("Input X must be a 3D numpy array with shape (N, N, L).")

    N, _, L = X.shape  # Number of nodes and layers
    Vs = []  # To store co-membership matrices from each layer
    combos_s = list(combinations(range(N), 2))  # Generate pairs of nodes

    for i in range(L):
        v = []  # Co-membership vector for the current layer
        net = X[:, :, i]  # Extract the i-th layer as a NumPy array

        # Create an igraph directed graph for each layer
        G = ig.Graph.Weighted_Adjacency(net.tolist(), mode="directed", attr="weight")
        partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

        M = partition.membership

        # Compute co-membership matrix for the current layer
        co_matrix = np.zeros((N, N))
        for ii in range(len(combos_s)):
            m1 = M[combos_s[ii][0]]
            m2 = M[combos_s[ii][1]]
            if m1 == m2:
                co_matrix[combos_s[ii][0], combos_s[ii][1]] = 1
                co_matrix[combos_s[ii][1], combos_s[ii][0]] = 1  # Ensure symmetry

        Vs.append(co_matrix)

    # Convert the list of co-membership matrices to a 3D array
    Vs = np.stack(Vs, axis=2)

    # Aggregate the co-membership matrices by summing across layers
    aggregated_vs = np.sum(Vs, axis=2)

    # Create a graph for the aggregated matrix (using undirected mode)
    # Aggregated co-membership matrix should be treated as undirected
    G = ig.Graph.Weighted_Adjacency(aggregated_vs.tolist(), mode="undirected", attr="weight")
    partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

    Q = partition.q  # Modularity score
    M = partition.membership

    # Return the optimal number of clusters and other results
    Opt_rank = max(M) + 1
    return Opt_rank, M, aggregated_vs, Q



In [ ]:
Opt_rank, M, aggregated_vs, Q = Divisive_Clustering(mat)
print("Optimal number of clusters:", Opt_rank)
print("Community assignments:", M)
print("Modularity score:", Q)
print("Aggregated co-membership matrix (Vs):\n", aggregated_vs)


Optimal number of clusters: 3
Community assignments: [0, 2, 2, 1, 1, 0, 0, 1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 1, 0, 0]
Modularity score: -0.03473684210526312
Aggregated co-membership matrix (Vs):
 [[ 0.  8.  6.  5. 10. 10.  9.  6.  9. 12.  2.  5.  8.  9.  6. 10.  6.  6.
  15. 13.]
 [ 8.  0. 14.  9.  8.  5.  2.  8. 15.  2.  7. 10. 12. 11.  3.  4. 13.  6.
   6.  7.]
 [ 6. 14.  0.  8.  5.  7.  5.  7. 15.  5.  9. 10.  7. 14.  5.  1. 20.  3.
   8.  4.]
 [ 5.  9.  8.  0.  9.  6.  9.  9.  7.  9. 15. 11.  7.  7.  9.  9.  9. 11.
   2.  3.]
 [10.  8.  5.  9.  0. 12.  6. 17.  5.  9.  8. 14. 10.  4.  6.  9.  5. 11.
   9. 11.]
 [10.  5.  7.  6. 12.  0. 11. 10.  6. 14.  6.  8.  8. 11.  9.  6.  7.  8.
  11.  9.]
 [ 9.  2.  5.  9.  6. 11.  0.  3.  6. 17.  7.  5.  4.  7.  8. 12.  5.  7.
   9.  9.]
 [ 6.  8.  7.  9. 17. 10.  3.  0.  6.  5. 11. 13.  9.  4.  6.  6.  8. 13.
   8.  8.]
 [ 9. 15. 15.  7.  5.  6.  6.  6.  0.  6.  5.  7. 11. 13.  5.  4. 15.  5.
   7.  6.]
 [12.  2.  5.  9.  9. 14. 17.  5.  6.  0.  

In [ ]:
def recursive_Divisive_Clustering(X, max_depth=3, current_depth=0):
    """
    Apply the community detection recursively and store the results in a tree structure.

    Arguments:
    X : 3D numpy array representing the graph layers.
    max_depth : Maximum recursion depth.
    current_depth : Current recursion depth (used for tracking).

    Returns:
    tree_structure : A tree-like dictionary of the community detection results or None if stopped.
    """
    # Stop recursion if the current depth exceeds the max_depth
    if current_depth >= max_depth:
        return None

    # Apply the consensus function to get results
    Opt_rank, M, aggregated_vs, Q = Divisive_Clustering(X)

    # If Opt_rank is 1, stop recursion and return None (this branch will not have any output)
    if Opt_rank == 1:
        return None

    # Prepare the tree structure at the current depth
    result_tree = {
        'Depth': current_depth,
        'Community Membership': M,
        'Modularity Score': Q,
        'Aggregated Co-membership Matrix': aggregated_vs,
    }

    # Create subgraphs based on community membership
    subgraphs = {}
    N = len(M)
    for community_id in set(M):
        # Create a subgraph for each community
        community_nodes = [i for i in range(N) if M[i] == community_id]
        if len(community_nodes) > 1:  # Only recurse on non-trivial subgraphs
            subgraph_data = X[community_nodes, :][:, community_nodes, :]
            subgraphs[community_id] = recursive_Divisive_Clustering(subgraph_data, max_depth, current_depth + 1)

    # Only add subgraphs if there are any, otherwise return None
    if subgraphs:
        result_tree['Subgraphs'] = subgraphs
    else:
        result_tree['Subgraphs'] = {}

    return result_tree



In [ ]:
# Example usage
# Assuming `mat` is your 3D numpy array (shape: N x N x L)
result_tree = recursive_Divisive_Clustering(mat, max_depth=4,current_depth=0)
result_tree

{'Depth': 0,
 'Community Membership': [0,
  2,
  2,
  1,
  1,
  0,
  0,
  1,
  2,
  0,
  1,
  1,
  1,
  2,
  0,
  0,
  2,
  1,
  0,
  0],
 'Modularity Score': -0.034473684210526254,
 'Aggregated Co-membership Matrix': array([[ 0.,  8.,  6.,  5., 10., 11., 10.,  7.,  8., 12.,  2.,  6.,  7.,
          9.,  6., 11.,  6.,  6., 15., 14.],
        [ 8.,  0., 14.,  9.,  8.,  4.,  3.,  9., 14.,  2.,  7., 11., 11.,
         11.,  4.,  4., 13.,  6.,  6.,  8.],
        [ 6., 14.,  0.,  8.,  7.,  6.,  5.,  6., 14.,  5.,  9., 10.,  6.,
         14.,  6.,  1., 20.,  3.,  8.,  5.],
        [ 5.,  9.,  8.,  0.,  7.,  5., 10., 10.,  8.,  9., 15., 12.,  8.,
          7., 10.,  9.,  9., 11.,  2.,  2.],
        [10.,  8.,  7.,  7.,  0., 11.,  5., 16.,  4.,  7.,  8., 13.,  9.,
          5.,  7.,  9.,  7.,  9., 10., 11.],
        [11.,  4.,  6.,  5., 11.,  0., 12.,  8.,  4., 15.,  5.,  7.,  8.,
         10., 10.,  8.,  6.,  9., 12., 11.],
        [10.,  3.,  5., 10.,  5., 12.,  0.,  4.,  8., 18.,  7.,  5., 